In [ ]:
import os
import pandas as pd
from IPython.display import display

# --- Configuração dos Diretórios ---
PROCESSED_PATH = os.path.join('data', 'processed')
OUTPUTS_PATH = os.path.join('data', 'outputs')
os.makedirs(OUTPUTS_PATH, exist_ok=True)

# --- Caminho do Arquivo Final ---
FINAL_PARQUET_PATH = os.path.join(PROCESSED_PATH, 'dados_b3.parquet')

print("--- CARREGANDO DATAFRAME DO PARQUET ---")

try:
    # Carrega o dataset principal, que é a fonte de toda a informação
    df_final = pd.read_parquet(FINAL_PARQUET_PATH)
    print(f"\n[SUCESSO] Dataset principal carregado com {len(df_final):,} registros.")
except FileNotFoundError:
    print(f"[ERRO FATAL] O arquivo '{FINAL_PARQUET_PATH}' não foi encontrado. Execute o processamento principal primeiro.")
    exit()
except Exception as e:
    print(f"[ERRO FATAL] Não foi possível carregar o dataset principal: {e}")
    exit()

In [ ]:
# ==============================================================================
# PRIMEIRO: Geração de Dicionários
# ==============================================================================
print("\n\n--- INICIANDO GERAÇÃO DE DICIONÁRIOS E MAPEAMENTOS ---")

# --- 1. Dicionário Master de Ativos (Security Master) ---
print("\n1. Criando o Dicionário Master de Ativos...")
try:
    # Usamos o df_final completo para garantir que nenhum ativo seja perdido
    df_ativos = df_final[['DATA_PREGAO', 'CODISI', 'CODNEG', 'NOMRES', 'ESPECI']].copy()
    df_ativos.dropna(subset=['CODISI'], inplace=True) # Dropar se o ISIN for nulo é seguro

    df_base_ativos = (
        df_ativos.sort_values('DATA_PREGAO', ascending=False)
        .drop_duplicates(subset=['CODISI'], keep='first')
        .rename(columns={'CODNEG': 'ULTIMO_TICKER', 'NOMRES': 'ULTIMO_NOME', 'ESPECI': 'ULTIMA_ESPECIFICACAO'})
        .drop(columns='DATA_PREGAO')
    )

    # --- CORREÇÃO AQUI ---
    # Modificamos a função lambda para remover valores nulos ANTES de tentar ordenar.
    # Usamos pd.notna() para filtrar e str() para garantir que tudo seja texto.
    df_variacoes_tickers = (
        df_ativos.groupby('CODISI')['CODNEG']
        .apply(lambda x: ' | '.join(sorted([str(item) for item in x.unique() if pd.notna(item)])))
        .reset_index()
        .rename(columns={'CODNEG': 'TICKERS_HISTORICOS'})
    )
    
    # --- E CORREÇÃO AQUI TAMBÉM ---
    df_variacoes_nomes = (
        df_ativos.groupby('CODISI')['NOMRES']
        .apply(lambda x: ' | '.join(sorted([str(item) for item in x.unique() if pd.notna(item)])))
        .reset_index()
        .rename(columns={'NOMRES': 'NOMES_HISTORICOS'})
    )

    df_dicionario_ativos = pd.merge(df_base_ativos, df_variacoes_tickers, on='CODISI', how='left')
    df_dicionario_ativos = pd.merge(df_dicionario_ativos, df_variacoes_nomes, on='CODISI', how='left')

    cols_ordem = [
        'CODISI', 'ULTIMO_TICKER', 'ULTIMO_NOME', 'ULTIMA_ESPECIFICACAO', 
        'TICKERS_HISTORICOS', 'NOMES_HISTORICOS'
    ]
    df_dicionario_ativos = df_dicionario_ativos[cols_ordem].sort_values('ULTIMO_TICKER').reset_index(drop=True)

    output_path_parquet = os.path.join(OUTPUTS_PATH, 'dicionario_ativos.parquet')
    output_path_excel = os.path.join(OUTPUTS_PATH, 'dicionario_ativos.xlsx')
    
    df_dicionario_ativos.to_parquet(output_path_parquet, index=False)
    df_dicionario_ativos.to_excel(output_path_excel, index=False)
    
    print(f" -> [SUCESSO] Dicionário com {len(df_dicionario_ativos)} ativos únicos gerado.")
    print(f"    -> Salvo em (Parquet): {output_path_parquet}")
    print(f"    -> Salvo em (Excel): {output_path_excel}")

except Exception as e:
    print(f" -> [ERRO] Falha ao gerar o Dicionário Master de Ativos: {e}")

# --- 2. Dicionários de Códigos da B3 ---
print("\n2. Gerando Dicionários de Códigos (CODBDI e TPMERC)...")
try:
    # Usei seu mapeamento completo aqui para ser mais robusto
    map_codbdi = {
        '02': 'LOTE PADRÃO', '05': 'SANCIONADAS', '06': 'CONCORDATÁRIAS', '07': 'RECUPERAÇÃO EXTRAJUDICIAL',
        '08': 'RECUPERAÇÃO JUDICIAL', '09': 'REGIME DE ADMINISTRAÇÃO ESPECIAL TEMPORÁRIA',
        '10': 'DIREITOS E RECIBOS', '11': 'INTERVENÇÃO', '12': 'FUNDOS IMOBILIÁRIOS', '14': 'CERTIFICADOS DE INVESTIMENTO',
        '18': 'OBRIGAÇÕES', '22': 'BÔNUS (PRIVADOS)', '26': 'APÓLICES/BÔNUS/TÍTULOS (PÚBLICOS)',
        '32': 'EXERCÍCIO DE OPÇÕES DE COMPRA DE ÍNDICES', '33': 'EXERCÍCIO DE OPÇÕES DE VENDA DE ÍNDICES',
        '38': 'EXERCÍCIO DE OPÇÕES DE COMPRA', '42': 'EXERCÍCIO DE OPÇÕES DE VENDA', '46': 'LEILÃO DE AÇÕES EM MORA',
        '48': 'LEILÃO DE AÇÕES (ART. 49)', '49': 'LEILÃO DE AÇÕES', '50': 'LEILÃO DE AÇÕES', '51': 'LEILÃO DE AÇÕES',
        '52': 'LEILÃO DE AÇÕES', '53': 'LEILÃO DE AÇÕES', '54': 'LEILÃO DE AÇÕES', '56': 'LEILÃO DE AÇÕES',
        '58': 'LEILÃO', '60': 'LEILÃO', '61': 'LEILÃO', '62': 'LEILÃO', '66': 'DEBÊNTURES COM DATA DE VENCIMENTO ATÉ 3 ANOS',
        '68': 'DEBÊNTURES COM DATA DE VENCIMENTO MAIOR QUE 3 ANOS', '70': 'FUTURO COM RETENÇÃO DE GANHOS',
        '71': 'FUTURO COM MOVIMENTAÇÃO DIÁRIA', '74': 'OPÇÕES DE COMPRA DE ÍNDICES', '75': 'OPÇÕES DE VENDA DE ÍNDICES',
        '78': 'OPÇÕES DE COMPRA', '82': 'OPÇÕES DE VENDA', '83': 'BOVESPAFIX', '84': 'SOMA FIX', '90': 'TERMO',
        '96': 'FRACIONÁRIO', '99': 'TOTAL'
    }
    df_dict_codbdi = pd.DataFrame(list(map_codbdi.items()), columns=['CODBDI', 'DESCRICAO_CODBDI'])
    
    map_tpmerc = {
        '010': 'VISTA', '012': 'EXERCÍCIO DE OPÇÕES DE COMPRA', '013': 'EXERCÍCIO DE OPÇÕES DE VENDA',
        '017': 'LEILÃO', '020': 'FRACIONÁRIO', '030': 'TERMO', '050': 'FUTURO COM RETENÇÃO DE GANHO',
        '060': 'FUTURO COM MOVIMENTAÇÃO DIÁRIA', '070': 'OPÇÕES DE COMPRA', '080': 'OPÇÕES DE VENDA'
    }
    df_dict_tpmerc = pd.DataFrame(list(map_tpmerc.items()), columns=['TPMERC', 'DESCRICAO_TPMERC'])

    path_codbdi = os.path.join(OUTPUTS_PATH, 'dicionario_codbdi.xlsx')
    path_tpmerc = os.path.join(OUTPUTS_PATH, 'dicionario_tpmerc.xlsx')
    
    df_dict_codbdi.to_excel(path_codbdi, index=False)
    df_dict_tpmerc.to_excel(path_tpmerc, index=False)
    
    print(f" -> [SUCESSO] Dicionário de CODBDI salvo em: {path_codbdi}")
    print(f" -> [SUCESSO] Dicionário de TPMERC salvo em: {path_tpmerc}")

except Exception as e:
    print(f" -> [ERRO] Falha ao gerar os Dicionários de Códigos: {e}")

print("\n--- GERAÇÃO DE ARQUIVOS AUXILIARES CONCLUÍDA ---")

In [ ]:
# ==============================================================================
# SEGUNDO: Verificação e Análise
# ==============================================================================
print("\n\n--- INICIANDO VERIFICAÇÃO DE INTEGRIDADE DOS DADOS ---")

# Para a análise, podemos criar um DataFrame limpo sem modificar o original
df_analise = df_final.copy()

# Listas de colunas para verificação
PRICE_COLS = ['PREABE', 'PREMAX', 'PREMIN', 'PREMED', 'PREULT', 'PREOFC', 'PREOFV', 'PREEXE', 'VOLTOT']
INT_COLS = ['TIPREG', 'CODBDI', 'TPMERC', 'TOTNEG', 'QUATOT', 'FATCOT', 'INDOPC', 'DISMES', 'PRAZOT', 'PTOEXE']

try:
    # --- LIMPEZA PÓS-PROCESSAMENTO ---
    initial_rows = len(df_analise)
    df_analise.dropna(subset=['NOMRES'], inplace=True)
    if len(df_analise) < initial_rows:
        print(f"Para fins de análise, foram removidos {initial_rows - len(df_analise):,} registros onde 'NOMRES' era nulo.")

    # --- 1. Verificação de Tipos de Dados ---
    print("\n1. Verificando os tipos de dados (Dtypes) das colunas:")
    display(df_analise.info(verbose=False))

    # --- 2. Verificação de Valores Nulos ---
    print("\n2. Verificando a presença de valores nulos por coluna:")
    missing_values = df_analise.isnull().sum()
    missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
    if not missing_values.empty:
        print("Colunas com valores nulos encontrados:")
        display(missing_values.to_frame('contagem_nulos'))
    else:
        print("[OK] Nenhuma coluna com valores nulos.")
        
    # --- 4. Análise Descritiva das Colunas Numéricas ---
    print("\n4. Análise estatística descritiva das principais colunas numéricas:")
    display(df_analise[PRICE_COLS + ['TOTNEG', 'QUATOT']].describe().style.format('{:,.2f}'))
    
    print("\n--- VERIFICAÇÃO DE INTEGRIDADE CONCLUÍDA ---")

except Exception as e:
    print(f"[ERRO] Ocorreu um erro inesperado ao carregar ou verificar os dados: {e}")

In [ ]:
import os
import pandas as pd
from IPython.display import display

# --- 1. Configuração dos Caminhos ---
PROCESSED_PATH = os.path.join('data', 'processed')
FINAL_PARQUET_PATH = os.path.join(PROCESSED_PATH, 'dados_b3.parquet')
ACOES_PARQUET_PATH = os.path.join(PROCESSED_PATH, 'historico_acoes.parquet')

print("--- INICIANDO FILTRAGEM PARA DATASET DE AÇÕES ---")

try:
    df_completo = pd.read_parquet(FINAL_PARQUET_PATH)
    print(f"\n[SUCESSO] Dataset completo carregado com {len(df_completo):,} registros.")
except Exception as e:
    print(f"[ERRO FATAL] Não foi possível carregar o dataset principal: {e}")
    # exit() # Descomente se estiver rodando como script .py

# --- 2. Definição dos Filtros para Ações (COM TIPOS CORRIGIDOS) ---
print("\n2. Aplicando filtros para isolar apenas ações de empresas...")

# CORREÇÃO: Usar números inteiros (int) em vez de textos (str) para corresponder ao tipo de dado no DataFrame.
filtro_codbdi = df_completo['CODBDI'].isin([2, 96])
filtro_tpmerc = df_completo['TPMERC'].isin([10, 20])

df_acoes = df_completo[filtro_codbdi & filtro_tpmerc].copy()

# --- 3. Relatório da Filtragem ---
registros_iniciais = len(df_completo)
registros_finais = len(df_acoes)
percentual_retido = (registros_finais / registros_iniciais) * 100 if registros_iniciais > 0 else 0

print(f" -> Registros iniciais: {registros_iniciais:,}")
print(f" -> Registros após filtrar por ações: {registros_finais:,}")
print(f" -> Percentual de dados retidos: {percentual_retido:.2f}%")

# --- 4. Seleção de Colunas Essenciais para ML ---
colunas_essenciais = [
    'DATA_PREGAO', 'CODISI', 'CODNEG', 'NOMRES', 'ESPECI',
    'PREABE', 'PREMAX', 'PREMIN', 'PREULT', 'VOLTOT', 'QUATOT', 'FATCOT'
]

df_acoes = df_acoes[colunas_essenciais]
print(f"\n3. Selecionadas {len(colunas_essenciais)} colunas essenciais para o dataset de Machine Learning.")

# --- 5. Salvando o Novo Dataset Filtrado ---
try:
    print(f"\n4. Salvando o dataset de ações em formato Parquet...")
    df_acoes.to_parquet(ACOES_PARQUET_PATH, index=False, compression='snappy')
    print(f" -> [SUCESSO] Dataset 'historico_acoes.parquet' salvo em: {ACOES_PARQUET_PATH}")
except Exception as e:
    print(f" -> [ERRO] Falha ao salvar o novo arquivo Parquet: {e}")

# --- 6. Verificação Final ---
print("\n--- Verificação Final do Dataset de Ações ---")
print("5 primeiras linhas do novo DataFrame 'df_acoes':")
display(df_acoes.tail())
print("\nInformações e tipos de dados do novo DataFrame:")
df_acoes.info()